## Train on all four modules
## Predict on 1 module at a time A,B,C,D

In [ ]:
# we know what the feature signature for each module is.
# grab feature signature for A
# extract rows from X_test, y_test that have same signature as A

In [16]:
def featureReader(featuresFile):
    #reads featurefiles and projects 3 columns (ppn, sizeGB, Ytime)
    #reads profiledFile (the table from profiling) and reads all 55 columns
    #generates one dataframe with 58 columns and n rows (=size of profiled data from csv)
    import pandas as pd
    from IPython.display import display

    header='ft1 ft2 ft3 ft4 ft5 ft6 ft7 ft8 ft9 ft10 ft11 ft12 ft13 ft14 ft15 ft16 ft17 \
    ft18 ft24 ft25 ft19 ft39 ft20 ft33 ft21 ft35 ft22 ft23 ft34 ft36 ft37 ft38 ft40 ft41 \
    ft42 ft43 ft44 ft45 ft46 ft48 ft47 ft49 ft51 ft50 ft52 ft53 ft54 ft55 ft26 ft27 ft28 ft29 ft30 ft31 ft32'
    
    def value(item):
        return item[item.find('=')+1:]
    
    print("Reading features from %s" % featuresFile)
    df_features = pd.read_table(featuresFile, header=None, delimiter=',',
                       converters={i:value for i in range(55)},
                       names=header.split())
    df_features = df_features.astype(float)
    return df_features

Reading features from ./features/cdhitdup_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2984.0,1149.0,1350.0,1.0,1794.0,748.0,140.0,865.0,833.0,244.0,...,74.0,2087.0,3.0,83.41,197.65,669.0,230.0,1783.0,26.0,873.0


In [103]:
import pickle
with open('datasetIPPD.pickle', 'rb') as handle:
    data = pickle.load(handle)

#extract X_test, Y_test
X_test = data['X_test']
y_test = data['y_test']

In [104]:
# Signature for A
from IPython.display import display
signatureA = featureReader('./features/cdhitdup_singlecmd_features.txt')
display(signatureA)

Reading features from ./features/cdhitdup_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2984.0,1149.0,1350.0,1.0,1794.0,748.0,140.0,865.0,833.0,244.0,...,74.0,2087.0,3.0,83.41,197.65,669.0,230.0,1783.0,26.0,873.0


In [105]:
# Signature for B
signatureB = featureReader('./features/frhit_singlecmd_features.txt')
display(signatureB)

Reading features from ./features/frhit_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2286.0,985.0,973.0,24.0,1436.0,577.0,113.0,778.0,543.0,166.0,...,104.0,2007.0,5.0,126.11,152.59,223.0,458.0,1445.0,0.0,681.0


## A=cdhitdup, B=frhit, C=velvetG, D=velvetH

In [106]:
# Signature for C
signatureC = featureReader('./features/velvetg_singlecmd_features.txt')
display(signatureC)

Reading features from ./features/velvetg_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,9837.0,4069.0,3972.0,78.0,6043.0,2139.0,633.0,3149.0,2417.0,637.0,...,429.0,6062.0,298.0,349.07,777.61,1330.0,1353.0,6132.0,67.0,2616.0


In [107]:
# Signature for D
signatureD = featureReader('./features/velveth_singlecmd_features.txt')
display(signatureD)

Reading features from ./features/velveth_singlecmd_features.txt


,ft1,ft2,ft3,ft4,ft5,ft6,ft7,ft8,ft9,ft10,...,ft53,ft54,ft55,ft26,ft27,ft28,ft29,ft30,ft31,ft32
0,2301.0,833.0,979.0,14.0,1428.0,503.0,132.0,631.0,628.0,160.0,...,89.0,1295.0,22.0,88.28,195.57,224.0,347.0,1492.0,13.0,558.0


## Extractor for extracting features

In [145]:
cols = ['ft1', 'ft2', 'ft3', 'ft4', 'ft5', 'ft6', 'ft7', 'ft8', 'ft9', 'ft10',
       'ft11', 'ft12', 'ft13', 'ft14', 'ft15', 'ft16', 'ft17', 'ft18', 'ft24',
       'ft25', 'ft19', 'ft39', 'ft20', 'ft33', 'ft21', 'ft35', 'ft22', 'ft23',
       'ft34', 'ft36', 'ft37', 'ft38', 'ft40', 'ft41', 'ft42', 'ft43', 'ft44',
       'ft45', 'ft46', 'ft48', 'ft47', 'ft49', 'ft51', 'ft50', 'ft52', 'ft53',
       'ft54', 'ft55', 'ft26', 'ft27', 'ft28', 'ft29', 'ft30', 'ft31', 'ft32']

In [146]:
def extractor(signature, X_test, y_test):
    import pandas as pd
    import numpy  as np
    count = 0
    xt = pd.DataFrame(columns = X_test.columns)
    yt = []
    
    for i in range(len(X_test)):
        if(all(X_test[cols].iloc[[i]].values[0] == signature.iloc[[0]].values[0])):
            count = count + 1
            xt = xt.append(X_test.iloc[[i]], ignore_index=True)
            yt.append(y_test[i])
            
    print("Total matches found : %d " % count)
    
    return xt,np.asarray(yt)

In [147]:
X_A, y_A = extractor(signatureA, X_test, y_test)
X_A.shape, len(y_A)

Total matches found : 5 


((5, 57), 5)

In [148]:
X_B, y_B = extractor(signatureB, X_test, y_test)
X_B.shape, len(y_B)

Total matches found : 13 


((13, 57), 13)

In [149]:
X_C, y_C = extractor(signatureC, X_test, y_test)
X_C.shape, len(y_C)

Total matches found : 23 


((23, 57), 23)

In [150]:
X_D, y_D = extractor(signatureD, X_test, y_test)
X_D.shape, len(y_D)

Total matches found : 24 


((24, 57), 24)

In [151]:
len(X_test)

65

In [152]:
# import all the filenames
from allfunctions import *

In [153]:
%load_ext autoreload
%autoreload 2

<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>
<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:><:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>:<:>


In [155]:
# Read (frome file) the tuned model for each of 10 Model types
# Apply each of the 10 *best* models to 4 pairs(X_A, y_A) ...
# Generate 4 graphs with 10 test RMSEs on them (train RMSE, test RMSE, mean test set value)

# 1 LassoCVModel

In [158]:
#A
y_A.mean()

117.83600000000004

In [159]:
#B
y_B.mean()

2189.0938461538458

In [160]:
#C
y_C.mean()

1703.2260869565218

In [161]:
#D
y_D.mean()

1307.5374999999999

# 2 OMPCVModel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 3 GradientBoostingCVModel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 4 Random Forest

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 5 RidgeCVModel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 6 ElasticNetCVModel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 7 SVR - a - Poly Kernel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 8 SVR - b - Sigmoid Kernel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 9 SVR - c - Linear Kernel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D

# 10 SVR - d - rbf Kernel

In [ ]:
#A

In [ ]:
#B

In [ ]:
#C

In [ ]:
#D